In [1]:
from pyspark import SparkConf, SparkContext
import sys


#Create a configuration object and
#set the name of the application
conf = SparkConf().setAppName("Spark Word Count")
# Create a Spark Context object
sc = SparkContext.getOrCreate(conf=conf)

In [2]:
folderPath = "covid_jrc_1_aug_2021.csv"
colnames = ['Date','iso3','CountryName','Region','lat','lon','CumulativePositive','CumulativeDeceased','CumulativeRecovered','CurrentlyPositive','Hospitalized','IntensiveCare','EUcountry','EUCPMcountry','NUTS']

In [3]:
logFileRDD = sc.textFile(folderPath)

In [4]:
"""def to_float(item):
    if len(item) > 0:
        item = float(item)
        return item
    return 0
    
class CovidInfo(object):
    def __init__(self, cum_positive, cum_deceased, cum_recovered, curr_positive, hospitalized, intensive_care,
                 iso3, country_name, EUCountry, EUCPMCountry):
        self.cum_positive, self.cum_deceased, self.cum_recovered = cum_positive, cum_deceased, cum_recovered
        self.curr_positive, self.hospitalized, self.intensive_care = curr_positive, hospitalized, intensive_care
        self.iso3, self.country_name, self.EUCountry, self.EUCPMCountry = iso3, country_name, EUCountry, EUCPMCountry
   

class DateContainer(object):
    def __init__(self):
        self.dates = []
       
        
    def add(self, date, row):
        row_data = row.split(",")
        date = row_data[0]
        iso3 = row_data[1]
        country_name = row_data[2]
        
        region = row_data[3]
        #lat = float(row_data[4])
        #long = float(row_data[5])
        
        
        cum_positive, cum_deceased, cum_recovered, curr_positive, hospitalized, intensive_care = [ to_float(item) for item in row_data[6:12] ]
        '''
        if len(row_data[6]) > 0:
            cum_positive = float(row_data[6])
        else:
            cum_positive = 0
        if len(row_data[7]) > 0:
            cum_deceased = float(row_data[7])
        else:
            
        if len(row_data[6]) > 0:
            cum_recovered = float(row_data[8])
        else:
            cum_recovered = 0
        if len(row_data[6]) > 0:
            curr_positive = float(row_data[9])
        else:
            curr_positive =0
        if len(row_data[6]) > 0:
            hospitalized = float(row_data[10])
        else:
            hospitalized = 0
        if len(row_data[6]) > 0:
            intensive_care = float(row_data[11])
        else:
            intensive_care = 0
        '''
        EUCountry = row_data[12]
        EUCPMCountry = row_data[13]
        
        NUTS = row_data[14]
        
        self.dates.append((date, CovidInfo(cum_positive, cum_deceased, cum_recovered, curr_positive, hospitalized, intensive_care,
                                           iso3, country_name, EUCountry, EUCPMCountry) )
                             )
"""

'def to_float(item):\n    if len(item) > 0:\n        item = float(item)\n        return item\n    return 0\n    \nclass CovidInfo(object):\n    def __init__(self, cum_positive, cum_deceased, cum_recovered, curr_positive, hospitalized, intensive_care,\n                 iso3, country_name, EUCountry, EUCPMCountry):\n        self.cum_positive, self.cum_deceased, self.cum_recovered = cum_positive, cum_deceased, cum_recovered\n        self.curr_positive, self.hospitalized, self.intensive_care = curr_positive, hospitalized, intensive_care\n        self.iso3, self.country_name, self.EUCountry, self.EUCPMCountry = iso3, country_name, EUCountry, EUCPMCountry\n   \n\nclass DateContainer(object):\n    def __init__(self):\n        self.dates = []\n       \n        \n    def add(self, date, row):\n        row_data = row.split(",")\n        date = row_data[0]\n        iso3 = row_data[1]\n        country_name = row_data[2]\n        \n        region = row_data[3]\n        #lat = float(row_data[4])\n  

In [26]:
from  mapred_utils import DateContainer, CovidData
#currently positive and hospitalized not needed (the latter has too many nans)

# map to (nuts,(date, row_info) )
def map0(row):
    row_data = row.split(",")
    date = row_data[0]
    nuts = row_data[-1]
    # avoid mapping the header
    if "CumulativePositive" not in row:
        return nuts, (date, CovidData.parse(row))
    return None
def reduce0(item1, item2):
    date1, covid1 = item1
    date2, covid2 = item2
    
    if date1 >= date2:
        return date1, CovidData.time_aggregate(covid1, covid2)
    else:
        return date2, CovidData.time_aggregate(covid1, covid2)

    
def map1(item):
    nuts, (date, covid_data) = item
    #row_data = row.split(",")
    #date = row_data[0]
    #nuts = row_data[-1]
    
    if len(nuts)> 4:
        nuts = nuts[:4]
    return nuts, covid_data
    
def reduce1(covid1, covid2):
    return CovidData.aggregate(covid1, covid2)
    
    

        
    
    
    
    

nogroup_count = logFileRDD.map(map0).filter(lambda item: item is not None)\
        .reduceByKey(reduce0)\
        .map(map1).filter(lambda item: item is not None)\
        .filter(lambda item: len(item[0]) ==4 )\
        .count()

group_count = logFileRDD.map(map0).filter(lambda item: item is not None)\
        .reduceByKey(reduce0)\
        .map(map1).filter(lambda item: item is not None)\
        .filter(lambda item: len(item[0]) ==4 )\
        .groupByKey()\
        .count()


In [27]:
print(nogroup_count)
print(group_count)

380
223


# find regions that at the latest date have no cases (missing data error, jrc side)

In [37]:
from  mapred_utils import DateContainer, CovidData
res = logFileRDD.map(map0).filter(lambda item: item is not None)\
        .reduceByKey(reduce0)\
        .map(map1).filter(lambda item: item is not None)\
        .filter(lambda item: len(item[0]) ==4 )\
        .reduceByKey(reduce1)\
        .filter(lambda item: item[1].cum_positive == 0.)\
        .map(lambda item: item[1].as_row(item[0]))\
        .collect()
res

['FRH0,0.0,1665.0,8139.0,-9804.0,279.0,22.0,FRA,France,True,True,2021-07-28',
 'FRM0,0.0,206.0,983.0,-1189.0,31.0,4.0,FRA,France,True,True,2021-07-28',
 'FRY3,0.0,175.0,3240.0,-3415.0,118.0,29.0,FRA,France,True,True,2021-07-28',
 'FRY4,0.0,282.0,2307.0,-2589.0,149.0,48.0,FRA,France,True,True,2021-07-28',
 'FRG0,0.0,2743.0,13412.0,-16155.0,151.0,21.0,FRA,France,True,True,2021-07-28',
 'FRY1,0.0,328.0,1619.0,-1947.0,128.0,22.0,FRA,France,True,True,2021-07-28',
 'FRB0,0.0,2789.0,12554.0,-15343.0,216.0,27.0,FRA,France,True,True,2021-07-28',
 'FRY2,0.0,115.0,912.0,-1027.0,148.0,30.0,FRA,France,True,True,2021-07-28',
 'FRY5,0.0,131.0,1274.0,-1405.0,0.0,0.0,FRA,France,True,True,2021-07-28']

# find all regions that can be used for analysis

In [39]:
from  mapred_utils import DateContainer, CovidData
res = logFileRDD.map(map0).filter(lambda item: item is not None)\
        .reduceByKey(reduce0)\
        .map(map1).filter(lambda item: item is not None)\
        .filter(lambda item: len(item[0]) ==4 )\
        .reduceByKey(reduce1)\
        .filter(lambda item: item[1].cum_positive > 0.)\
        .map(lambda item: item[1].as_row(item[0]))\
        .collect()
res

['ITC4,852193.0,33823.0,808143.0,10227.0,237.0,26.0,ITA,Italy,True,True,2021-07-30',
 'ITH3,435954.0,11641.0,412386.0,11927.0,119.0,14.0,ITA,Italy,True,True,2021-07-30',
 'ITH5,393632.0,13282.0,373860.0,6490.0,211.0,14.0,ITA,Italy,True,True,2021-07-30',
 'ITH1,73775.0,1183.0,72252.0,340.0,10.0,0.0,ITA,Italy,True,True,2021-07-30',
 'ITG1,242059.0,6043.0,226021.0,9995.0,298.0,30.0,ITA,Italy,True,True,2021-07-30',
 'AT11,18243.0,331.0,17823.0,89.0,0.0,1.0,AUT,Austria,True,True,2021-07-30',
 'AT22,81129.0,2095.0,78499.0,535.0,5.0,1.0,AUT,Austria,True,True,2021-07-30',
 'AT33,63929.0,688.0,62831.0,410.0,6.0,2.0,AUT,Austria,True,True,2021-07-30',
 'AT13,141536.0,2365.0,134850.0,4321.0,38.0,23.0,AUT,Austria,True,True,2021-07-30',
 'ITF1,76124.0,2514.0,72215.0,1395.0,22.0,0.0,ITA,Italy,True,True,2021-07-30',
 'ITF4,255799.0,6669.0,246971.0,2159.0,90.0,8.0,ITA,Italy,True,True,2021-07-30',
 'CH04,115961.0,1418.0,0.0,114543.0,45.0,14.0,CHE,Switzerland,False,False,2021-07-31',
 'ITF6,70833.0,1255.

In [1]:
import urllib.request

covid_data_url = "https://raw.githubusercontent.com/ec-jrc/COVID-19/master/data-by-region/jrc-covid-19-all-days-by-regions.csv"
    
with urllib.request.urlopen(covid_data_url) as f:
    string = f.read().decode('utf-8')
    


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

